# Using MDAnalysis to load trajectories in NanoVer

## Read the trajectory with MDAnalysis

We have a small trajectory of oseltamivir unbinding from neuraminidase, produced with iMD-VR:

In [1]:
topology_file = 'files/3TI6_ose_wt.pdb'
trajectory_file = 'files/ose_wt.dcd'

Construct an MDAnalysis universe from the topology and trajectory files:

In [2]:
import MDAnalysis as mda

universe = mda.Universe(topology_file, trajectory_file, trajectory=True, to_guess=("bonds",))
universe.trajectory

C:\Users\ragzo\anaconda3\envs\nanover-dev\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


<DCDReader files/ose_wt.dcd with 24 frames of 5877 atoms>

## Serve the trajectory with NanoVer



Make a NanoVer "simulation" from the MDAnalysis Universe:

In [3]:
from nanover.mdanalysis import UniverseSimulation

simulation = UniverseSimulation.from_universe(universe, name="Oseltamivir unbinding")
simulation.playback_factor = 30  # At 30fps this corresponds to 1 ps per s.

Start a NanoVer server with this simulation:

In [4]:
from nanover.app import OmniRunner

runner = OmniRunner.with_basic_server(simulation, port=0)
runner.load(0)

Use the NGLClient to check everything is working:

In [5]:
from nanover.jupyter import NGLClient

client = NGLClient.from_runner(runner)
client.view

NGLWidget()

C:\Users\ragzo\anaconda3\envs\nanover-dev\Lib\site-packages\MDAnalysis\coordinates\PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
C:\Users\ragzo\anaconda3\envs\nanover-dev\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1193: UserWarning: Found chainIDs with invalid length. Corresponding atoms will use value of 'X'
  warnings.warn("Found chainIDs with invalid length."


## Making it pretty in VR

Let's change how it looks. We've covered this in more detail in the [neuraminidase iMD](../ase/openmm_neuraminidase.ipynb) and [graphene](../ase/openmm_graphene.ipynb) examples.

**Note**: A lot of this will probably be made simpler in upcoming releases

In [6]:
from nanover.websocket import NanoverImdClient
client = NanoverImdClient.from_runner(runner)
client.wait_until_minimum_usable_frame();

In [7]:
import matplotlib.cm

def get_matplotlib_gradient(name: str):
    cmap = matplotlib.colormaps[name]
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

Check out the MDAnalysis documentation for [information about creating selections](https://userguide.mdanalysis.org/stable/selections.html)

In [8]:
from nanover.mdanalysis import frame_data_to_mdanalysis
def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.current_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

In [9]:
# Hide default rendering
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True
    root_selection.interaction_method = 'none'

In [10]:
# Create selection for the protein
protein = client.create_selection("Protein", [])
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H"))

In [11]:
# Set the rendering of this selection
with protein.modify():
    protein.renderer = {
            'sequence': 'polypeptide',
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('viridis')
            },
            'render': 'geometric spline',
            'scale': 0.2
        }

In [12]:
# Create selection for the ligand
ligand = client.create_selection("Ligand", [])
with ligand.modify():
    ligand.set_particles(generate_mdanalysis_selection("resname OSE"))

In [13]:
# Set the rendering of this selection
with ligand.modify():
    ligand.renderer = {
            'scale': 0.1,
            'render': 'liquorice',
            'color': 'cpk',
        }

## Tidying Up

In [14]:
runner.close()